In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import sys
sys.path.append('/home/jupyter-yehlin/Pairformer/boltzdesign')
from boltzdesign_utils import *
from ligandmpnn_utils import *

/home/jupyter-yehlin/.local/lib/python3.9/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
## ccd_library 
import pickle
import rdkit
filename = '/home/jupyter-yehlin/Pairformer/boltz/ccd.pkl'
ccd_lib = pickle.load(open(filename, 'rb'))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
predict_args = {
    "recycling_steps": 0,  # Default value
    "sampling_steps": 200,  # Default value
    "diffusion_samples": 1,  # Default value
    "write_confidence_summary": True,
    "write_full_pae": False,
    "write_full_pde": False,
}

boltz_model = get_boltz_model('/home/jupyter-yehlin/Pairformer/boltz/boltz1_conf.ckpt',predict_args,device)
boltz_model.train()

## Run BoltzDesign

In [5]:
##default config
config = {
    'mutation_rate': 1,
    'pre_iteration': 30,
    'soft_iteration': 75, 
    'temp_iteration': 45,
    'hard_iteration': 5,
    'semi_greedy_steps': 0,
    'learning_rate_pre': 0.2,
    'learning_rate': 0.1,
    'design_algorithm': '3stages',
    'set_train': True,
    'use_temp': True,
    'disconnect_feats': True,
    'disconnect_pairformer': False,
    'length': 150,
    'distogram_only': True,
    'binder_chain': 'A', ## A or B
    'small_molecule': True,
    'mask_ligand': False,
    'optimize_per_contact_per_binder_pos': True
}


In [6]:
yaml_dir = '/home/jupyter-yehlin/Pairformer/boltzdesign/examples/small_molecule'
for yaml_path in os.listdir(yaml_dir):
    if yaml_path.endswith('.yaml'):
        print(yaml_path)

8vhp.yaml
6CZI.yaml
1HXD.yaml
3WC0.yaml


In [7]:
# Set up directories
main_dir = '/home/jupyter-yehlin/Pairformer/boltzdesign/examples_test'
yaml_dir = '/home/jupyter-yehlin/Pairformer/boltzdesign/examples/small_molecule'
os.makedirs(main_dir, exist_ok=True)

version_name = 'small_molecule'
design_samples = 5

loss_scales = {
    'con_loss': 1.0,
    'i_con_loss': 1.0, 
    'helix_loss': -0.2,
    'plddt_loss': 0.1,
    'pae_loss': 0.4,
    'i_pae_loss': 0.1,
    'rg_loss': 0.4,
}

run_boltz_design(
    main_dir=main_dir,
    yaml_dir=yaml_dir, 
    boltz_model=boltz_model,
    ccd_lib=ccd_lib,
    design_samples=design_samples,
    version_name=version_name,
    config=config, 
    loss_scales=loss_scales
)

pre-run warm up
Found explicit empty MSA for some proteins, will run these in single sequence mode. Keep in mind that the model predictions will be suboptimal without an MSA.
set in train mode
Pre run- input_res_type tensor([[[ 0.0178,  0.0476, -0.0488,  ...,  0.1800,  0.1728,  0.1661],
         [ 0.2573,  0.2069, -0.0754,  ...,  0.0555,  0.0835,  0.0121],
         [-0.1713, -0.0057,  0.3168,  ..., -0.0761, -0.0728,  0.0073],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0')
Error processing 8vhp iteration 1: design() got an unexpected keyword argument 'optimize_per_contact_per_binder_pos'
pre-run warm up
Found explicit empty MSA for some proteins, will run these in single sequence mode. Keep in mind that the model predictions will be suboptimal without an MSA.
set in train mode
Pre run

## 2. LigandMPNN Redesign

In [ ]:
main_dir = '/home/jupyter-yehlin/Pairformer/boltzdesign/examples/small_molecule'
yaml_dir = '/home/jupyter-yehlin/Pairformer/boltzdesign/examples/small_molecule'
boltzdesign_dir = main_dir + '/results_final'
pdb_save_dir = main_dir + '/pdb'
ligandmpnn_dir = main_dir + '/ligandmpnn'

os.makedirs(ligandmpnn_dir, exist_ok=True)
convert_cif_files_to_pdb(boltzdesign_dir, pdb_save_dir)
run_ligandmpnn_redesign(ligandmpnn_dir, pdb_save_dir, yaml_dir, top_k=1, cutoff=6, non_protein_ligand=True, binder_chain='A', target_chain='B')


Converting /home/jupyter-yehlin/Pairformer/boltzdesign/examples/rfdiffusion_small_molecule/results_final/boltz_results_7v11_results_itr1_length150/predictions/7v11_results_itr1_length150/7v11_results_itr1_length150_model_0.cif
Converting /home/jupyter-yehlin/Pairformer/boltzdesign/examples/rfdiffusion_small_molecule/results_final/boltz_results_5sdv_results_itr1_length150/predictions/5sdv_results_itr1_length150/5sdv_results_itr1_length150_model_0.cif
Converting /home/jupyter-yehlin/Pairformer/boltzdesign/examples/rfdiffusion_small_molecule/results_final/boltz_results_7bkc_results_itr1_length150/predictions/7bkc_results_itr1_length150/7bkc_results_itr1_length150_model_0.cif


DEBUG: 1352 atoms and 1 coordinate set(s) were parsed in 0.01s.


len interface_residues 14


DEBUG: 1352 atoms and 1 coordinate set(s) were parsed in 0.01s.


Designing protein from this path: /home/jupyter-yehlin/Pairformer/boltzdesign/examples/rfdiffusion_small_molecule/pdb/7v11_results_itr1_length150_model_0.pdb
fixed_residues ['A31', 'A32', 'A33', 'A34', 'A64', 'A67', 'A76', 'A82', 'A83', 'A86', 'A87', 'A90', 'A120', 'A123']
These residues will be redesigned:  ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'A65', 'A66', 'A68', 'A69', 'A70', 'A71', 'A72', 'A73', 'A74', 'A75', 'A77', 'A78', 'A79', 'A80', 'A81', 'A84', 'A85', 'A88', 'A89', 'A91', 'A92', 'A93', 'A94', 'A95', 'A96', 'A97', 'A98', 'A99', 'A100', 'A101', 'A102', 'A103', 'A104', 'A105', 'A106', 'A107', 'A108', 'A109', 'A110', '

...

Predicting DataLoader 0: 100%|██████████| 1/1 [00:08<00:00,  0.12it/s]


DEBUG: 1368 atoms and 1 coordinate set(s) were parsed in 0.03s.
DEBUG: 1368 atoms and 1 coordinate set(s) were parsed in 0.01s.


Completed processing 5sdv_results_itr1_length150_model_0 for sequence 1
len interface_residues 8
Designing protein from this path: /home/jupyter-yehlin/Pairformer/boltzdesign/examples/rfdiffusion_small_molecule/pdb/7bkc_results_itr1_length150_model_0.pdb
fixed_residues ['A50', 'A53', 'A116', 'A117', 'A134', 'A135', 'A136', 'A137']
These residues will be redesigned:  ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A51', 'A52', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63', 'A64', 'A65', 'A66', 'A67', 'A68', 'A69', 'A70', 'A71', 'A72', 'A73', 'A74', 'A75', 'A76', 'A77', 'A78', 'A79', 'A80', 'A81', 'A82', 'A83', 'A84', 'A85', 'A86', 'A87', 'A88', 'A89', 'A90', 'A91', 'A92', 'A93', 'A

/home/jupyter-yehlin/.local/lib/python3.9/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


KeyboardInterrupt: 

In [32]:
import json
import glob
import os
import numpy as np

ligandmpnn_dir_boltz = ligandmpnn_dir + '/boltz_predictions_success_lmpnn'
for root in os.listdir(ligandmpnn_dir_boltz):
    root_path = os.path.join(ligandmpnn_dir_boltz, root, 'predictions')
    for subdir in os.listdir(root_path):
        json_path = os.path.join(root_path, subdir, f'confidence_{subdir}_model_0.json')
        pae_path = os.path.join(root_path, subdir, f'pae_{subdir}_model_0.npz')

        length = int(subdir[subdir.find('length')+6:subdir.find('_model')])
        
        if os.path.exists(json_path):
            with open(json_path, 'r') as f:
                data = json.load(f)
            design_name = json_path.split('/')[-2]
            try:
                pae_data = np.load(pae_path)
                pae_matrix = pae_data['pae']
                interface_pae = np.mean(((pae_matrix + pae_matrix.T)/2)[:length,length:])
                print(np.mean(pae_matrix[length:,:length]))
                print(np.mean(pae_matrix[:length,length:]))
                intra_pae = np.mean(pae_matrix[:length,:length])
                print(f"{design_name} length: {length} complex_plddt: {data['complex_plddt']:.2f} iptm: {data['iptm']:.2f} i-pae: {interface_pae:.2f} pae: {intra_pae:.2f}")
            except KeyError:
                print(f"{design_name} length: {length} complex_plddt: {data['complex_plddt']:.2f} iptm: {data['iptm']:.2f} PAE data not found")

20.980047
5.9006677
7v11_results_itr1_length150_model_0_1 length: 150 complex_plddt: 0.82 iptm: 0.61 i-pae: 13.44 pae: 2.93
17.219994
4.119095
5sdv_results_itr1_length150_model_0_1 length: 150 complex_plddt: 0.86 iptm: 0.80 i-pae: 10.67 pae: 3.13
19.964653
7.0432525
7bkc_results_itr1_length150_model_0_1 length: 150 complex_plddt: 0.76 iptm: 0.75 i-pae: 13.50 pae: 7.94
